In [1]:
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D , MaxPool2D, Flatten, Dense, Conv2D, Dropout, Convolution2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16

import warnings
import glob
import cv2

import numpy as np
from tensorflow.keras.preprocessing import image
warnings.filterwarnings('ignore')

In [2]:
DATASET_PATH = 'datasets/weapon_dataset_v2'

image_height = 64
image_width = 64
classes_list = [ 'normal', 'weapon']

In [3]:
vgg = VGG16(input_shape=[image_height, image_width] + [3], weights='imagenet', include_top=False)

In [4]:
for layer in vgg.layers:
    layer.trainable = False

In [5]:
x = Flatten()(vgg.output)

In [6]:
prediction = Dense(1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [8]:
# model = Sequential()
    
# # Layer: 1
# model.add(Conv2D(filters = 16, kernel_size = 3, padding='same',activation = 'relu', input_shape = (image_height, image_width, 3)))
# model.add(MaxPool2D(4,4))
# model.add(Dropout(0.25))

# # Layer: 2
# model.add(Conv2D(filters= 32, kernel_size=3,padding='same', activation='relu'))
# model.add(MaxPool2D(4,4))
# model.add(Dropout(0.25))

# # Layer: 3
# model.add(Conv2D(filters= 64, kernel_size=3,padding='same', activation='relu'))
# model.add(MaxPool2D(2,2))
# model.add(Dropout(0.25))

# # Layer: 4
# model.add(Conv2D(filters= 64, kernel_size=3,padding='same', activation='relu'))
# model.add(MaxPool2D(2,2))
# model.add(Dropout(0.25))

# # Flatten Layer
# model.add(Flatten())

# # Final Classification Layer
# model.add(Dense(1, activation='sigmoid'))

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
training_set = train_datagen.flow_from_directory(
                            DATASET_PATH + '/training_set',
                            target_size=(image_width, image_height),
                            class_mode='binary')

Found 354 images belonging to 2 classes.


In [11]:
test_set = train_datagen.flow_from_directory(
                            DATASET_PATH + '/testing_set',
                            target_size=(image_width, image_height),
                            class_mode='binary')

Found 58 images belonging to 2 classes.


In [12]:
# Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
 
# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 
# Start Training
model.fit_generator(training_set,
         steps_per_epoch=len(training_set),
         epochs=25,
         validation_data=test_set,
         validation_steps=len(test_set),
        callbacks = [early_stopping_callback])

Epoch 1/25
12/12 [==============================] - 66s 5s/step - loss: 0.6179 - accuracy: 0.6723 - val_loss: 0.6048 - val_accuracy: 0.6897
Epoch 2/25
12/12 [==============================] - 44s 4s/step - loss: 0.4515 - accuracy: 0.7571 - val_loss: 0.4955 - val_accuracy: 0.7586
Epoch 3/25
12/12 [==============================] - 36s 3s/step - loss: 0.3744 - accuracy: 0.8842 - val_loss: 0.4734 - val_accuracy: 0.7759
Epoch 4/25
12/12 [==============================] - 35s 3s/step - loss: 0.3074 - accuracy: 0.8898 - val_loss: 0.4628 - val_accuracy: 0.7759
Epoch 5/25
12/12 [==============================] - 35s 3s/step - loss: 0.2806 - accuracy: 0.9124 - val_loss: 0.4107 - val_accuracy: 0.8276
Epoch 6/25
12/12 [==============================] - 35s 3s/step - loss: 0.2564 - accuracy: 0.9492 - val_loss: 0.3921 - val_accuracy: 0.8448
Epoch 7/25
12/12 [==============================] - 35s 3s/step - loss: 0.2244 - accuracy: 0.9294 - val_loss: 0.4109 - val_accuracy: 0.8103
Epoch 8/25
12/12 [==

In [16]:
model.save('models/WEAPON_MODEL_64.h5')

In [17]:
# model = load_model('models/WEAPON_MODEL_128.h5')

In [19]:
test_image = image.load_img('testing_data/weapon.jpg', target_size=(image_width, image_height))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict(test_image)
print(prediction)

1/1 [==============================] - 0s 216ms/step
[[0.894908]]
